In [ ]:
import pandas as pd
import pandas_ta as pta
import matplotlib.pyplot as plt
import numpy as np

class PandasTAUtils:
    """Utilidades para calcular y visualizar indicadores técnicos usando Pandas TA"""
    
    @staticmethod
    def sma(prices, period=20):
        """Calcula la Media Móvil Simple (SMA)"""
        return pta.sma(prices, length=period)
    
    @staticmethod
    def ema(prices, period=20):
        """Calcula la Media Móvil Exponencial (EMA)"""
        return pta.ema(prices, length=period)
    
    @staticmethod
    def macd(prices, fast_period=12, slow_period=26, signal_period=9):
        """Calcula MACD (Moving Average Convergence/Divergence)"""
        macd_result = pta.macd(prices, fast=fast_period, slow=slow_period, signal=signal_period)
        macd = macd_result[f'MACD_{fast_period}_{slow_period}_{signal_period}']
        signal = macd_result[f'MACDs_{fast_period}_{slow_period}_{signal_period}']
        hist = macd_result[f'MACDh_{fast_period}_{slow_period}_{signal_period}']
        return macd, signal, hist
    
    @staticmethod
    def rsi(prices, period=14):
        """Calcula el índice de fuerza relativa (RSI)"""
        return pta.rsi(prices, length=period)
    
    @staticmethod
    def bollinger_bands(prices, period=20, num_std=2):
        """Calcula las Bandas de Bollinger"""
        bb_result = pta.bbands(prices, length=period, std=num_std)
        upper = bb_result[f'BBU_{period}_{num_std}.0']
        middle = bb_result[f'BBM_{period}_{num_std}.0']
        lower = bb_result[f'BBL_{period}_{num_std}.0']
        return upper, middle, lower
    
    @staticmethod
    def stochastic(high, low, close, k_period=14, d_period=3, smooth_k=3):
        """Calcula el Oscilador Estocástico"""
        stoch = pta.stoch(high, low, close, k=k_period, d=d_period, smooth_k=smooth_k)
        k = stoch[f'STOCHk_{k_period}_{smooth_k}_{d_period}']
        d = stoch[f'STOCHd_{k_period}_{smooth_k}_{d_period}']
        return k, d
    
    @staticmethod
    def adx(high, low, close, period=14):
        """Calcula el Índice de Movimiento Direccional Promedio (ADX)"""
        adx_result = pta.adx(high, low, close, length=period)
        return adx_result[f'ADX_{period}']
    
    @staticmethod
    def atr(high, low, close, period=14):
        """Calcula el Rango Promedio Verdadero (ATR)"""
        return pta.atr(high, low, close, length=period)
    
    @staticmethod
    def detect_candlestick_patterns(open_prices, high, low, close):
        """Detecta patrones de velas comunes usando Pandas TA"""
        patterns = {}
        
        # Lista de patrones a detectar
        pattern_names = [
            'doji', 'hammer', 'shootingstar', 'hangingman', 'invertedhammer',
            'engulfing', 'harami', 'morningstar', 'eveningstar'
        ]
        
        # Crear un DataFrame con los datos OHLC
        df = pd.DataFrame({
            'open': open_prices,
            'high': high,
            'low': low,
            'close': close
        })
        
        # Detectar cada patrón
        for pattern in pattern_names:
            try:
                result = pta.cdl_pattern(df['open'], df['high'], df['low'], df['close'], name=[pattern])
                pattern_key = pattern.capitalize()
                patterns[pattern_key] = result
            except:
                # Si el patrón no está disponible, continuar con el siguiente
                continue
        
        # Para patrones compuestos como Three White Soldiers que tienen nombres específicos en TA-Lib
        special_patterns = {
            'Three White Soldiers': 'threewhitesoldiers',
            'Three Black Crows': 'threeblackcrows'
        }
        
        for display_name, pattern_name in special_patterns.items():
            try:
                result = pta.cdl_pattern(df['open'], df['high'], df['low'], df['close'], name=[pattern_name])
                patterns[display_name] = result
            except:
                continue
        
        return patterns
    
    @staticmethod
    def plot_sma(prices, periods=[20, 50, 200], figsize=(12, 6)):
        """Grafica el precio y múltiples SMAs"""
        plt.figure(figsize=figsize)
        plt.plot(prices, label='Precio', alpha=0.7)
        
        for period in periods:
            sma = PandasTAUtils.sma(prices, period)
            plt.plot(sma, label=f'SMA {period}')
        
        plt.title('Precio y Medias Móviles Simples (SMA) - Pandas TA')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
    
    @staticmethod
    def plot_macd(prices, fast_period=12, slow_period=26, signal_period=9, figsize=(12, 6)):
        """Grafica el MACD"""
        macd, signal, hist = PandasTAUtils.macd(prices, fast_period, slow_period, signal_period)
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, gridspec_kw={'height_ratios': [3, 1]})
        
        # Graficar precios
        ax1.plot(prices)
        ax1.set_title('Precio')
        ax1.grid(True, alpha=0.3)
        
        # Graficar MACD
        ax2.plot(macd, label='MACD')
        ax2.plot(signal, label='Señal')
        ax2.bar(np.arange(len(hist)), hist, label='Histograma', alpha=0.5)
        ax2.axhline(y=0, color='r', linestyle='-', alpha=0.3)
        ax2.set_title('MACD - Pandas TA')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        plt.tight_layout()
        plt.show()